# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

## 1- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

### Almacenar los datos como dataframe

####He importado la libreria de crealytics para leer el xlsx > com.crealytics:spark-excel_2.12:3.2.0_0.16.0

In [0]:
df_xlsx = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true").option("inferSchema", "true") \
    .load("dbfs:/FileStore/Caso_3_Examen/airbnb_room_type.xlsx")

df_xlsx.printSchema()


root
 |-- listing_id: double (nullable = true)
 |-- description: string (nullable = true)
 |-- room_type: string (nullable = true)



##### CSV y TSV

In [0]:
df_tsv = spark.read.format("csv") \
    .option("delimiter", "\t") \
    .option("header", "true") \
    .load("dbfs:/FileStore/Caso_3_Examen/airbnb_last_review.tsv")

df_tsv.printSchema()

df_csv = spark.read.format("csv") \
    .option("header", "true") \
    .load("dbfs:/FileStore/Caso_3_Examen/airbnb_price.csv")

df_csv.printSchema()


root
 |-- listing_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- last_review: string (nullable = true)

root
 |-- listing_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- nbhood_full: string (nullable = true)



### Primera y ultimas fechas

In [0]:
from pyspark.sql.functions import col, to_date, min, max

# Especificar el formato de la fecha que estamos utilizando: 'Month Day Year' (e.g., 'June 22 2019')
date_format = 'MMMM dd yyyy'

# Convertir la columna 'last_review' al tipo Date usando el formato adecuado
df_tsv = df_tsv.withColumn("last_review_date", to_date(col("last_review"), date_format))

# Obtener la fecha de la primera y última reseña
first_review_date = df_tsv.select(min("last_review_date")).first()[0]
last_review_date = df_tsv.select(max("last_review_date")).first()[0]

# Imprimir las fechas
print("Fecha de la primera reseña:", first_review_date)
print("Fecha de la última reseña:", last_review_date)


Fecha de la primera reseña: 2019-01-01
Fecha de la última reseña: 2019-07-09


## 2- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

####Ver los tipos de habitaciones

In [0]:
# Obtener los valores únicos en la columna 'room_type'
unique_room_types = df_xlsx.select("room_type").distinct().collect()

# Imprimir los valores únicos
for row in unique_room_types:
    print(row["room_type"])


entire home/apt
Shared room
private room
ENTIRE HOME/APT
PRIVATE ROOM
Entire home/apt
Private room
SHARED ROOM
shared room


####Como el formato varía para las mismas opciones, hay que unificar para que salgan todas en lowercase

In [0]:
from pyspark.sql.functions import lower

# Convertir la columna 'room_type' a minúsculas
df_min = df_xlsx.withColumn("room_type", lower(df_xlsx["room_type"]))

# Mostrar el resultado
df_min.display()


listing_id,description,room_type
2595.0,Skylit Midtown Castle,entire home/apt
3831.0,Cozy Entire Floor of Brownstone,entire home/apt
5099.0,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
5178.0,Large Furnished Room Near B'way,private room
5238.0,Cute & Cozy Lower East Side 1 bdrm,entire home/apt
5295.0,Beautiful 1br on Upper West Side,entire home/apt
5441.0,Central Manhattan/near Broadway,private room
5803.0,"Lovely Room 1, Garden, Best Area, Legal rental",private room
6021.0,Wonderful Guest Bedroom in Manhattan for SINGLES,private room
6848.0,Only 2 stops to Manhattan studio,entire home/apt


#### Ahora buscar el private room


In [0]:
# Filtrar los anuncios que son de tipo 'private room' y contar cuántos hay
private_rooms_count = df_min.filter(col("room_type") == "private room").count()

print("Número de anuncios de habitaciones privadas:", private_rooms_count)


Número de anuncios de habitaciones privadas: 11356


## 3- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

#### Crear una nueva columna para sacar el numero de precio, ya que el formato de la columna es "225 dollars"

In [0]:
from pyspark.sql.functions import col, regexp_replace, avg, round

# Eliminar los textos no numéricos (como "dollars") y convertir a tipo numérico
df_csv = df_csv.withColumn("Precio", regexp_replace(col("price"), "[^0-9.]", "").cast("double"))
df_csv.limit(2).display()

listing_id,price,nbhood_full,Precio
2595,225 dollars,"Manhattan, Midtown",225.0
3831,89 dollars,"Brooklyn, Clinton Hill",89.0


####Sacar el promedio de esa columna nueva

In [0]:
# Calcular el precio promedio y redondearlo a 2 decimales
avg_price = df_csv.select(round(avg("Precio"), 2)).first()[0]

print("Precio promedio:", avg_price)


Precio promedio: 141.78


## 4- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql import Row

# Crear una fila con las variables obtenidas
review_dates = spark.createDataFrame([
    Row(first_reviewed=first_review_date, last_reviewed=last_review_date, nb_private_rooms=private_rooms_count, avg_price=avg_price)
])

# Mostrar el DataFrame final
review_dates.display()


first_reviewed,last_reviewed,nb_private_rooms,avg_price
2019-01-01,2019-07-09,11356,141.78
